# Installing and Importing the required Libraries

In [7]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Scraping the Wikipedia page for the table of postal codes of Canada

In [8]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


### Remove "\n" at the end of each string in the Neighborhood column

In [30]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.dropna(axis=0)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn


##### Group all neighborhoods with the same postal code

In [32]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.dropna(axis=0)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn


In [33]:
print("Shape: ", df.shape)

Shape:  (180, 3)


# Get the latitude and the longitude coordinates of each neighborhood.

In [34]:
df_geo_coor = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### We need to couple 2 dataframes "df" and "df_geo_coor" into one dataframe.

In [50]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.drop("PostalCode", axis=1, inplace=True)

#df_toronto.dropna(subset=["Postal Code", axis=0, inplace=True]
df_toronto.dropna(subset=["Latitude"], axis=0, inplace=True)
df_toronto.dropna(subset=["Longitude"], axis=0, inplace=True) 
                  
# reset index, because we droped
df_toronto.reset_index(drop=True, inplace=True)
                  
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


### And make sure that the dataset has all boroughs and neighborhoods

In [51]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


## Use geopy library to get the latitude and longitude values of Toronto

### In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [48]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


## Create a map of New York with neighborhoods superimposed on top

In [49]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#map_toronto

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Solving environment: done

# All requested packages already installed.



###### However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [53]:
Scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


#### Let's get the geographical coordinates of Scarborough.

In [54]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


#### let's visualizat Scarborough the neighborhoods in it.

In [55]:
# create map of Manhattan using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [56]:
CLIENT_ID = 'X1X04UPNBZWQKQYAGKUW0ISPBOD1YHSR2BYFS4TYGOXFNCCA' # your Foursquare ID
CLIENT_SECRET = 'YNL41N25BJEMQOP0KH1ZE41PRYGLZUIEOKFGR4WS1HH4XBYN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X1X04UPNBZWQKQYAGKUW0ISPBOD1YHSR2BYFS4TYGOXFNCCA
CLIENT_SECRET:YNL41N25BJEMQOP0KH1ZE41PRYGLZUIEOKFGR4WS1HH4XBYN


## Let's explore the first neighborhood in our dataframe.

#### Get the neighborhood's name.

In [58]:
Scarborough_data.loc[0, 'Neighborhood']

'Malvern / Rouge'

#### Get the neighborhood's latitude and longitude values.

In [59]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern / Rouge are 43.806686299999996, -79.19435340000001.


## Now, let's get the top 100 venues that are in Malvern / Rouge within a radius of 500 meters.

#### First, let's create the GET request URL. Name your URL url.

In [60]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
#-->

'https://api.foursquare.com/v2/venues/explore?&client_id=X1X04UPNBZWQKQYAGKUW0ISPBOD1YHSR2BYFS4TYGOXFNCCA&client_secret=YNL41N25BJEMQOP0KH1ZE41PRYGLZUIEOKFGR4WS1HH4XBYN&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

#### Send the GET request and examine the resutls

In [61]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eab345b9c6f59001b920c29'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

####  we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function.

In [62]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a pandas dataframe.

In [63]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


##### And how many venues were returned by Foursquare?

In [64]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


# Explore Neighborhoods in Scarborough

### Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called Scarborough_venues.

In [66]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge


#### Let's check the size of the resulting dataframe

In [67]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(92, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Guildwood / Morningside / West Hill,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


#### Let's check how many venues were returned for each neighborhood

In [68]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Birch Cliff / Cliffside West,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
Clarks Corners / Tam O'Shanter / Sullivan,13,13,13,13,13,13
Cliffside / Cliffcrest / Scarborough Village West,2,2,2,2,2,2
Dorset Park / Wexford Heights / Scarborough Town Centre,5,5,5,5,5,5
Golden Mile / Clairlea / Oakridge,10,10,10,10,10,10
Guildwood / Morningside / West Hill,7,7,7,7,7,7
Kennedy Park / Ionview / East Birchmount Park,6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues

In [69]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 57 uniques categories.


# Analyze Each Neighborhood

In [70]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Guildwood / Morningside / West Hill,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### And let's examine the new dataframe size.

In [71]:
Scarborough_onehot.shape

(92, 58)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [73]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.250000,0.0000,0.0,...,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.0000,0.000000,0.000000,0.0
1,Birch Cliff / Cliffside West,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.0000,0.000000,0.000000,0.0
2,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.125000,0.000000,0.0
3,Clarks Corners / Tam O'Shanter / Sullivan,0.0,0.000,0.000000,0.000000,0.076923,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.076923,0.000000,0.0
4,Cliffside / Cliffcrest / Scarborough Village West,0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.000000,0.000000,0.0
5,Dorset Park / Wexford Heights / Scarborough To...,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.000000,0.000000,0.2
6,Golden Mile / Clairlea / Oakridge,0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.0000,0.2,...,0.000000,0.000000,0.000000,0.00,0.000000,0.1,0.0000,0.000000,0.000000,0.0
7,Guildwood / Morningside / West Hill,0.0,0.000,0.000000,0.000000,0.142857,0.0,0.142857,0.0000,0.0,...,0.142857,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.000000,0.000000,0.0
8,Kennedy Park / Ionview / East Birchmount Park,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.000000,0.166667,0.0
9,Malvern / Rouge,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.000000,0.000000,0.0


#### Let's confirm the new size

In [74]:
Scarborough_grouped.shape

(16, 58)

#### Let's print each neighborhood along with the top 5 most common venues

In [75]:
num_top_venues = 5

for hood in Scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


----Birch Cliff / Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0      Hakka Restaurant  0.12
1  Caribbean Restaurant  0.12
2                Bakery  0.12
3                  Bank  0.12
4       Thai Restaurant  0.12


----Clarks Corners / Tam O'Shanter / Sullivan----
                 venue  freq
0          Pizza Place  0.15
1             Pharmacy  0.15
2         Noodle House  0.08
3   Italian Restaurant  0.08
4  Fried Chicken Joint  0.08


----Cliffside / Cliffcrest / Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1  

### Let's put that into a pandas dataframe

#### First, let's write a function to sort the venues in descending order.

In [95]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [143]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,Birch Cliff / Cliffside West,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Gym,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
3,Clarks Corners / Tam O'Shanter / Sullivan,Pharmacy,Pizza Place,Noodle House,Bank,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant
4,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Motel,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


# Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [144]:
# set number of clusters
kclusters = 5

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 4, 1, 1, 1, 1, 2], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [145]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged = Scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#.....
Scarborough_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)
                                

Scarborough_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,Malvern / Rouge,43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,0.0,Bar,Vietnamese Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,1.0,Bank,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Convenience Store,General Entertainment
3,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Insurance Office,Korean Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Electronics Store,Department Store


#### Finally, let's visualize the resulting clusters

In [146]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighborhood'], Scarborough_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



# Examine Clusters

##### Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [147]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Rouge Hill / Port Union / Highland Creek,Bar,Vietnamese Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster 2

In [148]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Guildwood / Morningside / West Hill,Bank,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Convenience Store,General Entertainment
3,Woburn,Coffee Shop,Insurance Office,Korean Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
6,Kennedy Park / Ionview / East Birchmount Park,Discount Store,Coffee Shop,Hobby Shop,Department Store,Train Station,Bank,Bar,Hakka Restaurant,Gym,Grocery Store
7,Golden Mile / Clairlea / Oakridge,Bus Line,Bakery,Ice Cream Shop,Intersection,Park,Bus Station,Metro Station,Soccer Field,Fried Chicken Joint,Convenience Store
9,Birch Cliff / Cliffside West,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Gym,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
10,Dorset Park / Wexford Heights / Scarborough To...,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Bakery,Convenience Store,Athletics & Sports,Grocery Store,General Entertainment,Gas Station
11,Wexford / Maryvale,Middle Eastern Restaurant,Auto Garage,Bakery,Smoke Shop,Shopping Mall,Sandwich Place,Breakfast Spot,Electronics Store,Convenience Store,Cosmetics Shop
12,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
13,Clarks Corners / Tam O'Shanter / Sullivan,Pharmacy,Pizza Place,Noodle House,Bank,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant


#### Cluster 3

In [149]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Malvern / Rouge,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


#### Cluster 4

In [150]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 3, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
14,Milliken / Agincourt North / Steeles East / L'...,Park,Playground,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Cluster 5

In [151]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 4, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Motel,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
